In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
from natsort import natsorted
from PIL import Image

import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision import datasets
import pytorch_lightning as pl
import torchmetrics
from torchmetrics.functional import accuracy
import torchsummary
from torchsummary import summary
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.models import efficientnet_b4, efficientnet_b7

In [3]:
# ネジの画像ファイルがある手前のパス
work_dir = os.getcwd()
# 訓練用画像、テスト用画像のパスをそれぞれ
train_path = os.path.join(work_dir, 'images_screw_data/train')
test_path = os.path.join(work_dir, 'images_screw_data/test')

# efficientnet_b7用
transform = transforms.Compose([
transforms.Resize(380),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# データセットを作成
train_val = datasets.ImageFolder(train_path, transform)
test = datasets.ImageFolder(test_path, transform)
# train, valのデータ総数
n_train_val = len(train_val)

# ハイパーパラメータの宣言
batch_size = 40
train_ratio = 0.7
val_ratio = round(1 - train_ratio, 1)

# データ分割数の設定 整数にしないとエラー出る
n_train = int(n_train_val * train_ratio)
n_val = n_train_val - n_train
n_list = [n_train, n_val]


# train, valに分割
pl.seed_everything(0)
train, val = torch.utils.data.random_split(train_val, n_list)

# DataLoader作成
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

Global seed set to 0


In [4]:
n_list

[140, 61]

In [4]:
class Net(pl.LightningModule):

    def __init__(
        self,
        feature=efficientnet_b4(pretrained=True),
        drop_rate_1=0.45,
        n_mid = 128,
        lr=0.000864,
        optimizer='Adagrad'
    ):
        super().__init__()
        
        self.feature = feature
        for param in self.feature.parameters():
            param.requires_grad = False
        
        self.drop_rate_1 = drop_rate_1
        self.n_mid = n_mid
        # self.drop_rate_2 = drop_rate_2
        self.lr = lr
        self.optimizer = optimizer

        self.fc1 = nn.Linear(1000, self.n_mid)
        self.drop1 = nn.Dropout2d(self.drop_rate_1)
        self.fc2 = nn.Linear(self.n_mid, 2)

    def forward(self, x):
        h = self.feature(x)
        h = self.fc1(h)
        h = self.drop1(h)
        h = self.fc2(h)
        # h = self.drop2(h)
        # h = self.fc3(h)
        return h


    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', accuracy(y.softmax(dim=-1), t), on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss


    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = getattr(torch.optim, self.optimizer)(self.parameters(), lr=self.lr)
        return optimizer

/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
def objective(trial):
    # パラメータの候補領域
    drop_rate_1 =  trial.suggest_discrete_uniform('drop_rate_1', 0.3, 0.9, 0.05)
    n_mid = trial.suggest_categorical('n_mid', [128, 256])
    # drop_rate_2 =  trial.suggest_discrete_uniform('drop_rate_2', 0.1, 0.6, 0.05)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-3)
    # optimizer = trial.suggest_categorical('optimizer', ['SGD', 'Adagrad', 'Adam',])
    
    # ネットワークの訓練（早期終了あり）
    pl.seed_everything(0)
    net = Net(
        drop_rate_1=drop_rate_1,
        n_mid = n_mid,
        # drop_rate_2=drop_rate_2,
        lr=lr,
        # optimizer=optimizer
    )
    
    #deteministic=Trueにすると、エラーが出てしまう（原因は解析してない）
    trainer = pl.Trainer(
        max_epochs=50,
        accelerator='gpu',
        devices='auto',
        # callbacks=[EarlyStopping(monitor='val_acc')]
    )
    trainer.fit(net, train_loader, val_loader)
    result = trainer.callback_metrics['val_acc']
    # 検証データの accuracy で評価
    return result

In [ ]:
# studyオブジェクトによる最適化
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=50)

[I 2022-11-21 13:47:51,397] A new study created in memory with name: no-name-2cd4440b-b6f9-4b4e-af05-7e0d9fe595fc
/tmp/ipykernel_26/1942162538.py:3: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  drop_rate_1 =  trial.suggest_discrete_uniform('drop_rate_1', 0.3, 0.9, 0.05)
/tmp/ipykernel_26/1942162538.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-3)
Global seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
print(study.best_value)
print(study.best_params)